In [1]:
from keras.models import Sequential
from keras.layers import MaxPool2D , Conv2D , Flatten , Dense , MaxPooling2D ,Dropout


C:\Users\Giott\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = Sequential()

In [3]:
model.add(Conv2D(filters=15 , kernel_size= 2 , padding='valid' , activation='relu' , input_shape=(200,200,1)))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 199, 199, 15)      75        
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________


In [5]:

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(units=128 , activation='relu'))
model.add(Dropout(0.4))
model.add(Dense (units=1  , activation='sigmoid'))
model.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'])
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
___________________________

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 64)          0         
__________

In [7]:
import keras as k 
import pandas as pd 
import numpy as np 
import matplotlib.pylab as plt 
%matplotlib inline
from keras.preprocessing import image
from keras.preprocessing.image import  ImageDataGenerator

In [8]:
parasite_datagen = image.ImageDataGenerator(rescale= 1./255 , shear_range=0.2 , zoom_range=0.2 , horizontal_flip=True)

In [9]:
parasite_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
uninfected_datagen = ImageDataGenerator(rescale=1./255)
parasite_data = parasite_datagen.flow_from_directory('images/cell_images',
                                                     target_size=(32,32),
                                                     batch_size=32,
                                                     class_mode = 'binary')
uninfected_data = uninfected_datagen.flow_from_directory('images/cell_images',
                                                        target_size=(32,32),
                                                        batch_size=32,
                                                        class_mode = 'binary')

Found 27558 images belonging to 2 classes.
Found 27558 images belonging to 2 classes.


In [10]:
from keras.callbacks import ModelCheckpoint  
import tensorflow as tf

In [11]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [12]:
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)
#using GPU
with tf.device('/gpu:0'):
    trained_model = model.fit_generator(parasite_data , steps_per_epoch=100,epochs=30 ,callbacks=[checkpointer] , validation_data=uninfected_data  , validation_steps=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
100/100 [==============================] - 18s 179ms/step - loss: 0.6472 - acc: 0.6272 - val_loss: 0.5949 - val_acc: 0.6856

Epoch 00001: val_loss improved from inf to 0.59489, saving model to model.weights.best.hdf5
Epoch 2/30
100/100 [==============================] - 16s 164ms/step - loss: 0.5727 - acc: 0.7159 - val_loss: 0.5114 - val_acc: 0.7775

Epoch 00002: val_loss improved from 0.59489 to 0.51144, saving model to model.weights.best.hdf5
Epoch 3/30
100/100 [==============================] - 16s 163ms/step - loss: 0.4689 - acc: 0.7991 - val_loss: 0.4988 - val_acc: 0.7575

Epoch 00003: val_loss improved from 0.51144 to 0.49881, saving model to model.weights.best.hdf5
Epoch 4/30
100/100 [==============================] - 16s 164ms/step - loss: 0.3882 - acc: 0.8438 - val_loss: 0.2757 - val_acc: 0.8794

Epoch 00004: val_loss improved from 0.49881 to 0.27571, saving model to model.weights.best.hdf5
Epoch 5/30
100/100 [========

In [21]:
acc = pd.DataFrame.from_dict(trained_model.history)
acc = pd.concat([pd.Series(range(0,30),name='epochs'),acc],axis=1)
acc.head(30)

,epochs,val_loss,val_acc,loss,acc
0,0,0.594888,0.685625,0.647182,0.627188
1,1,0.511436,0.777500,0.572717,0.715938
2,2,0.498813,0.757500,0.468892,0.799063
3,3,0.275714,0.879375,0.388176,0.843750
4,4,0.183887,0.927500,0.273034,0.899375
5,5,0.162549,0.945625,0.222117,0.921250
6,6,0.154458,0.951875,0.217715,0.928125
7,7,0.159055,0.950000,0.193333,0.932187
8,8,0.162708,0.950625,0.179529,0.940769
9,9,0.155762,0.951875,0.205054,0.934688
